# Tutorial

We now illustrate the basic capabilities of the ``respy`` package in a simple tutorial. The tutorial covers the following topics:

1. [How to specify a model - parameters and options.](#How-to-specify-a-model---parameters-and-options.)
2. [How to solve a model.](#How-to-solve-a-model.)
3. [How to simulate a data set.](#How-to-simulate-a-data-set.)
4. [How to estimate model parameters.](#How-to-estimate-model-parameters.)

After the [installation](installation.rst), the convention is to import ``respy`` as follows to get access to all exposed functions.

In [1]:
import respy as rp

## How to specify a model - parameters and options.

In order to define a model, ``respy`` relies on two objects - parameters and options. The difference between the two is that parameters are affected by the optimization whereas options stay the same. Here, only some parameters and options are explained. For a complete overview on parameters and options, visit the section [model specification](../software/model-specification.rst).

As an example, we turn to the first model of Keane and Wolpin (1994). The following function returns the parameters and options and the original dataset used by the authors. Passing ``with_data=False`` prevents the function to return a data set.

In [2]:
params, options = rp.get_example_model("kw_94_one", with_data=False)

As said before, options are everything not affected by the optimization. The value under ``"n_periods"`` defines the number of periods in the model meaning the available time frame in which individual can make their decisions.

The rest of the options includes seeds, numbers of draws to simulate the flow utilities in each period and other model parameters. Furthermore, there is a lot of logic hidden from the user to offer a comprehensible starting point. For a deeper understanding of the mechanics, the user is refered to the section [model specfication](../software/model-specification.rst).

In [3]:
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 1,
 'core_state_space_filters': ["period > 0 and exp_{i} == period and lagged_choice_1 != '{i}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{j}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{k}' and exp_{k} == 0",
  "period == 0 and lagged_choice_1 == '{k}'"],
 'covariates': {'constant': '1',
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2',
  'at_least_twelve_exp_edu': 'exp_edu >= 12',
  'not_edu_last_period': "lagged_choice_1 != 'edu'",
  'edu_ten': 'exp_edu == 10'}}

The parameter specification includes all parameters of the model which are affected by the optimization routine and some more.

In [4]:
params

value  \
category            name                                  
delta               delta                        0.9500   
wage_a              constant                     9.2100   
                    exp_edu                      0.0380   
                    exp_a                        0.0330   
                    exp_a_square                -0.0005   
                    exp_b                        0.0000   
                    exp_b_square                 0.0000   
wage_b              constant                     8.4800   
                    exp_edu                      0.0700   
                    exp_b                        0.0670   
                    exp_b_square                -0.0010   
                    exp_a                        0.0220   
                    exp_a_square                -0.0005   
nonpec_edu          constant                     0.0000   
                    at_least_twelve_exp_edu      0.0000   
                    not_edu_last_period      -4000.0000   
nonpec_home         constant                 17750.0000   
shocks_sdcorr       sd_a                         0.2000   
                    sd_b                         0.2500   
                    sd_edu                    1500.0000   
                    sd_home                   1500.0000   
                    corr_b_a                     0.0000   
                    corr_edu_a                   0.0000   
                    corr_edu_b                   0.0000   
                    corr_home_a                  0.0000   
                    corr_home_b                  0.0000   
                    corr_home_edu                0.0000   
lagged_choice_1_edu edu_ten                      1.0000   
initial_exp_edu     10                           1.0000   
maximum_exp         edu                         20.0000   

                                                                                       comment  
category            name                                                                        
delta               delta                                                      discount factor  
wage_a              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_a                                     return to same sector experience  
                    exp_a_square                   return to same sector, quadratic experience  
                    exp_b                                    return to other sector experience  
                    exp_b_square                  return to other sector, quadratic experience  
wage_b              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_b                                     return to same sector experience  
                    exp_b_square                   return to same sector, quadratic experience  
                    exp_a                                    return to other sector experience  
                    exp_a_square                  return to other sector, quadratic experience  
nonpec_edu          constant                            constant reward for choosing education  
                    at_least_twelve_exp_edu        reward for going to college (tuition, etc.)  
                    not_edu_last_period                        reward for going back to school  
nonpec_home         constant                         constant reward of non-market alternative  
shocks_sdcorr       sd_a                     Element 1,1 of standard-deviation/correlation ...  
                    sd_b                     Element 2,2 of standard-deviation/correlation ...  
                    sd_edu                   Element 3,3 of standard-deviation/correlation ...  
                    sd_home

## How to solve a model.

The solution of the model is the value of each state within the model assuming optimal decision in this and future periods. In ``respy``, the solution is represented with the :class:`~respy.state_space.StateSpace`, a class including the model solution. The solution combines the following steps:

- Building the state space.
- Solving the model by finding optimal decision in each state via backward induction.

The ``state_space`` is normally hidden from the user and only an internal construct. For a better understanding of how modeling works and for debugging, the state space is exposed to the user by :func:`~respy.solve.solve`.

In [5]:
state_space = rp.solve(params, options)

## How to simulate a data set.

The first step for simulating data is to build the simulation function. Then, pass the parameter vector to the function and retrieve a simulated data set.

In [6]:
simulate = rp.get_simulate_func(params, options)

In [7]:
df = simulate(params)

The step of building a simulate function may seem odd at first, but it has two advantages. First, this function can be easily used for estimation with simulated method of moments. An optimizer passes a new parameter vector to the function and receives data for matching moments. Second, while building the function, auxiliary components are created and attached to ``simulate()`` via :func:`functools.partial`. Thus, repetitive calls will be much faster. But, to simulate a different model where not only parameter values have changed, you need to build a new ``simulate()`` function.

## How to estimate model parameters.

To estimate model parameters via maximum likelihood, ``respy`` relies on [``estimagic``](https://github.com/OpenSourceEconomics/estimagic), an open-source tool to estimate structural models and more. First, we load the model again, because now we need the data. Actually, the data is the same as the previously simulated data.

In [8]:
params, options, df = rp.get_example_model("kw_94_one")

Second, we need the criterion function of the model which returns the likelihood value of the data given the model.

In [9]:
crit_func = rp.get_crit_func(params, options, df)

The criterion function takes the parameters as an input and returns the likelihood value. Depending on your computer the evaluation of the likelihood takes 1-3 seconds. In the optimization process, this routine is repeated over and over again for some thousand different parameter vectors.

In [10]:
%time crit_val = crit_func(params)
crit_val

Wall time: 1.89 s


-381.5728797655344

To estimate the model parameters, import the :func:`~estimagic.optimization.optimize.minimize` from ``estimagic``.

In [11]:
import numpy as np
from estimagic.optimization.optimize import maximize

For ``estimagic``, we need to pass constraints on the parameters in a list containing dictionaries. Each dictionary is a constraint. A constraint includes two components: First, we need to tell ``estimagic`` which parameters we want to constrain. This is achieved by specifying an index location which will be passed to `df.loc` or `df.query`. Then, define the type of the constraint. First, we impose the constraint that the shock parameters have to be valid variances and correlations. The last three contraints ensures that all individuals start with ten years of schooling, being in school the last period and that they can only ten additional years in school.

In [11]:
constr = rp.get_parameter_constraints("kw_94_one")
constr

C:\Users\tobia\git\respy\respy\conditional_draws.py:80: RuntimeWarning: overflow encountered in exp
  draws[:, :n_wages] = np.clip(np.exp(draws[:, :n_wages]), 0.0, HUGE_FLOAT)


[{'loc': 'shocks_sdcorr', 'type': 'sdcorr'},
 {'loc': 'lagged_choice_1_edu', 'type': 'fixed'},
 {'loc': 'initial_exp_edu', 'type': 'fixed'},
 {'loc': 'maximum_exp', 'type': 'fixed'}]

Optionally, we can add a column ``"group"`` which is identical to the category column. The estimagic dashboard will then contain one parameter convergence plot per group instead of plotting all parameters in the same figure. Since respy has quite many parameters, this will make the plots much more readable.

In [12]:
params["group"] = params.index.get_level_values('category')

In [13]:
results, params = maximize(
    crit_func,
    params,
    "scipy_L-BFGS-B",
    db_options={"rollover": 200},
    algo_options={"maxfun": 1},
    constraints=constr,
    dashboard=True
)

C:\Users\tobia\git\respy\respy\state_space.py:555: RuntimeWarning: overflow encountered in exp
  wages = np.clip(np.exp(log_wages), 0.0, HUGE_FLOAT)
C:\Users\tobia\git\respy\respy\likelihood.py:296: RuntimeWarning: divide by zero encountered in simulate_log_probability_of_individuals_observed_choice
  options["estimation_tau"],


You do not have to worry about warnings. During the estimation, the optimizer might try parameters which are to extreme and lead to infinite values or NaNs.

The estimation runs for approximately 90 evaluations and then stops because of ``"maxfun"``.

Look at the results by choosing the parameter vector or detailed information in the ``results`` dictionary.

Look at the results by choosing the parameter vector or detailed information in the ``results`` dictionary.

In [18]:
results.keys()

dict_keys(['fun', 'internal_x'])

In [19]:
results["fun"]

-9.999999999999998e+19

## References

> Keane, M. P. and  Wolpin, K. I. (1994). [The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence](https://doi.org/10.2307/2109768). *The Review of Economics and Statistics*, 76(4): 648-672.